<a href="https://colab.research.google.com/github/PN1019/StockExchanges/blob/main/AirFlowPipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting Up Stocks_Analysis Pipeline
 

*   Install and configure Airflow


*  setup official Apache/Airflow container with PostgreSQL and LocalExecutor using Docker and Docker Compose
*   List item


*   List item



In [1]:
!mkdir ~/airflow/

In [2]:
!mkdir ~/airflow/dags

In [11]:
!mkdir ~/airflow/scripts

In [9]:
! airflow db init

DB: sqlite:////root/airflow/airflow.db
[2021-07-23 17:54:46,176] {db.py:692} INFO - Creating tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
/usr/local/lib/python3.7/dist-packages/psycopg2/__init__.py:144 UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
INFO  [alembic.runtime.migration] Running upgrade  -> e3a246e0dc1, current schema
INFO  [alembic.runtime.migration] Running upgrade e3a246e0dc1 -> 1507a7289a2f, create is_encrypted
/usr/local/lib/python3.7/dist-packages/alembic/ddl/sqlite.py:44 UserWarning: Skipping unsupported ALTER for creation of implicit constraintPlease refer to the batch mode feature which allows for SQLite migrations using a copy-and-move strategy.
INFO  [alembic.runtime.migrati

In [10]:
!pip install docker-compose

     |████████████████████████████████| 114 kB 7.7 MB/s 
     |████████████████████████████████| 67 kB 3.1 MB/s 
     |████████████████████████████████| 146 kB 13.2 MB/s 
     |████████████████████████████████| 206 kB 11.5 MB/s 
     |████████████████████████████████| 63 kB 1.4 MB/s 
     |████████████████████████████████| 961 kB 13.3 MB/s 
  Created wheel for dockerpty: filename=dockerpty-0.4.1-py3-none-any.whl size=16615 sha256=6df3b722f1bc6dac0eae8b052c2d8ad54e5411246fa7e0edc197379b702f3969
  Stored in directory: /root/.cache/pip/wheels/0b/40/ea/b275182c42dc7be184c36518d42999bc5267268af2639e409b
Successfully built dockerpty


In [4]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 11.5 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=8044014498e1d853256769fd2a5f54e5acce66317eac9f457d498dd9dd373e1a
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [5]:
!pip install apache-airflow 

     |████████████████████████████████| 5.2 MB 8.2 MB/s 
     |████████████████████████████████| 636 kB 41.1 MB/s 
     |████████████████████████████████| 3.8 MB 38.4 MB/s 
     |████████████████████████████████| 3.2 MB 39.0 MB/s 
     |████████████████████████████████| 164 kB 41.6 MB/s 
     |████████████████████████████████| 208 kB 55.2 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 76 kB 5.4 MB/s 
     |████████████████████████████████| 155 kB 62.1 MB/s 
     |████████████████████████████████| 133 kB 59.6 MB/s 
     |████████████████████████████████| 79 kB 7.4 MB/s 
     |████████████████████████████████| 548 kB 40.8 MB/s 
     |████████████████████████████████| 1.8 MB 38.5 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 
     |████████████████████████████████| 111 kB 58.1 MB/s 
     |████████████████████████████████| 1.3 MB 34.7 MB/s 
     |███████████████████

In [6]:
from __future__ import absolute_import
from airflow import DAG
from airflow.operators.python import PythonOperator

from datetime import datetime, timedelta  
import datetime as dt
import pandas as pd
import yfinance as yf
import requests
import lxml
from functools import reduce


In [13]:
pip install --upgrade jupyter_http_over_ws>=0.0.7 && \jupyter serverextension enable --py jupyter_http_over_ws


Traceback (most recent call last):
  File "/usr/local/bin/jupyter-serverextension", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 267, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/notebook/serverextensions.py", line 293, in start
    super(ServerExtensionApp, self).start()
  File "/usr/local/lib/python2.7/dist-packages/jupyter_core/application.py", line 256, in start
    self.subapp.start()
  File "/usr/local/lib/python2.7/dist-packages/notebook/serverextensions.py", line 210, in start
    self.toggle_server_extension_python(arg)
  File "/usr/local/lib/python2.7/dist-packages/notebook/serverextensions.py", line 199, in toggle_server_extension_python
    m, server_exts = _get_server_extension_meta

In [ ]:
# Read and print the stock tickers that make up S&P500
tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
print(tickers.head())
start = dt.datetime.today()-dt.timedelta(2190) #Time before Six Years from Today
end = dt.datetime.today()

# Get the data for this tickers from yahoo finance
TICKER_LIST = [yf.download(tickers.Symbol.to_list(),start=start,end=end,auto_adjust=True)]
#print(data.head())


[2021-07-23 11:35:17,974] {utils.py:157} INFO - NumExpr defaulting to 2 threads.
  Symbol             Security  ...      CIK      Founded
0    MMM                   3M  ...    66740         1902
1    ABT  Abbott Laboratories  ...     1800         1888
2   ABBV               AbbVie  ...  1551152  2013 (1888)
3   ABMD              Abiomed  ...   815094         1981
4    ACN            Accenture  ...  1467373         1989

[5 rows x 9 columns]
[*********************100%***********************]  505 of 505 completed

2 Failed downloads:
- BRK.B: No data found, symbol may be delisted
- BF.B: No data found for this date range, symbol may be delisted
